In [1]:
import pandas as pd
import numpy as np
import pgeocode
import geopy
import random

In [2]:
def gerar_lista_ajustada(n_escolas, n_candidatos):
    """Gerando uma lista aleatória para substituir a quantidade de vagas para cada escola, de acordo com o numero de candidatos."""
    # Cria uma lista inicial com valores aleatórios
    lista = [(n_candidatos // n_escolas) for _ in range(n_escolas)]
    # Ajusta a soma da lista para que seja igual a n_candidatos
    soma_atual = sum(lista)
    diferenca = n_candidatos - soma_atual
    # Ajuste da lista para balancear a diferença
    while diferenca != 0:
        for i in range(n_escolas):
            # Ajuste gradual: adiciona ou subtrai 1 conforme a diferença restante
            if diferenca > 0 and lista[i] < 200:
                lista[i] += 1
                diferenca -= 1
            elif diferenca < 0 and lista[i] > 50:
                lista[i] -= 1
                diferenca += 1
            # Sai do loop se a diferença já for ajustada
            if diferenca == 0:
                break
    return lista

## Notebook para gerar samples de `Candidatos` e `Escolas`

### Abrindo os dados

In [3]:
# abrindo os dados dos candidatos em csv
df_candidatos = pd.read_csv('../dados/candidatos_concurso_raw.csv', sep=',')
print(df_candidatos.shape)
# agrupando caso tenha algum cep duplicado, pegando o primeiro nome
df_candidatos = df_candidatos.groupby('CEP').first().reset_index()
print(df_candidatos.shape)
df_candidatos.head()

(100000, 2)
(60302, 2)


,CEP,Nome
0,20010000,Zoe Abreu
1,20010010,Mariane Marques
2,20010020,Maria Vitória Camargo
3,20010040,Nicolas Dias
4,20010050,Luan Guerra


In [4]:
# abrindo os dados dos locais de prova em csv
df_escolas = pd.read_csv('../dados/escolas_finais_raw.csv', sep=',')
print(df_escolas.shape)
# agrupando caso tenha algum cep duplicado, pegando a primeira escola
df_escolas = df_escolas.groupby('CEP').first().reset_index()
print(df_escolas.shape)
df_escolas.head()

(358, 3)
(358, 3)


,CEP,Nome_Escola,Vagas
0,11111111,ESCOLA MUNICIPAL DR. KINGSTON GUIMARAES DE SOU...,277
1,20240200,ESCOLA MUNICIPAL GUATEMALA,274
2,20241220,COLÉGIO ANGELORUM,273
3,20241263,COLÉGIO ESTADUAL MONTEIRO DE CARVALHO,283
4,20251460,ESCOLA MUNICIPAL SANTA CATARINA,281


In [5]:
ceps_latlong = pd.read_csv("../dados/dataset_ceps_latlong_uf.csv")
print(ceps_latlong.shape)

# filtrando o UF do rio de janeiro
ceps_latlong = ceps_latlong[ceps_latlong['UF'] == 'RJ']
# groupby para pegar a primeira lat e long caso tenha algum cep repetido
ceps_latlong = ceps_latlong.groupby('CEP').first().reset_index().drop(columns=['UF'])
print(ceps_latlong.shape)

ceps_latlong.head()

(59939, 4)
(59723, 3)


,CEP,latitude,longitude
0,20010000,-22.901867,-43.176230
1,20010010,-22.902554,-43.174102
2,20010020,-22.904806,-43.174795
3,20010040,-22.820949,-43.376921
4,20010050,-22.904294,-43.173506


### Merge Candidatos e Escolas

In [7]:
# numero de amostras de candidatos e de escolas por sample
DATASET_SAMPLES = {
    1: (10, 5),
    2: (100, 25),
    3: (1000, 50),
    4: (10000, 100),
}

# iterando pelos datasets e numeros de escolas e candidatos
for dataset in DATASET_SAMPLES.keys():

    candidatos = df_candidatos.merge(ceps_latlong, on='CEP')

    # rename para Latitude e Londitude
    candidatos = candidatos.rename(columns={'latitude': 'Latitude', 'longitude': 'Longitude'})

    # removendo os ERROs
    candidatos = candidatos[candidatos['Latitude'] != 'ERRO']
    candidatos = candidatos.dropna()
    # convertendo para float
    candidatos['Latitude'] = candidatos['Latitude'].astype(float)
    candidatos['Longitude'] = candidatos['Longitude'].astype(float)
    candidatos = candidatos.groupby('CEP').first().reset_index()

    # Fazendo um sample nos candidatos
    n = DATASET_SAMPLES[dataset][0]
    candidatos = candidatos.sample(n).reset_index(drop=True)

    # Salvando os dados
    candidatos.to_csv(f"../dados/{dataset}_candidatos_{n}.csv", index=False)

    #---------------------------------------------------------------------------
    escolas = df_escolas.merge(ceps_latlong, on='CEP')
    escolas = escolas.drop_duplicates(subset='Nome_Escola')
    escolas = escolas.dropna()

    # filtrar todos os com nome "ERRO" na coluande latitude
    escolas = escolas[escolas['latitude'] != "ERRO"]

    # passando apra float
    escolas['latitude'] = escolas['latitude'].astype(float)
    escolas['longitude'] = escolas['longitude'].astype(float)

    # sample das escolas
    n = DATASET_SAMPLES[dataset][1]
    escolas = escolas.sample(n).reset_index(drop=True)
    
    n_candidatos = candidatos.shape[0]
    print("Candidatos:", n_candidatos)

    n_escolas = escolas.shape[0]
    print("Escolas:", n_escolas)
    
    lista_ajustada = gerar_lista_ajustada(n_escolas, n_candidatos)
    print("Lista ajustada:", lista_ajustada)
    print("Soma da lista:", sum(lista_ajustada))

    # Inserindo a quantidaed de vagas nas escolas
    escolas['Vagas'] = lista_ajustada
    # validação
    print("validação:", escolas['Vagas'].sum() == n_candidatos)
    
    # rename das latitudes e longitudes das escolas
    escolas = escolas.rename(columns={'latitude': 'Latitude', 'longitude': 'Longitude'}) 
    # salvando as escolas em csv
    escolas.to_csv(f"../dados/{dataset}_escolas_{n}.csv", index=False)

    print(f"Datasets salvos com sucesso!")
    print("-----------------------------------------------------")


Candidatos: 10
Escolas: 5
Lista ajustada: [2, 2, 2, 2, 2]
Soma da lista: 10
validação: True
Datasets salvos com sucesso!
-----------------------------------------------------
Candidatos: 100
Escolas: 25
Lista ajustada: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
Soma da lista: 100
validação: True
Datasets salvos com sucesso!
-----------------------------------------------------
Candidatos: 1000
Escolas: 50
Lista ajustada: [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
Soma da lista: 1000
validação: True
Datasets salvos com sucesso!
-----------------------------------------------------
Candidatos: 10000
Escolas: 100
Lista ajustada: [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,

***
***